In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7814005a-6586-b9f3-f37b-e8d0249a4e0a)


## Download Dataset

In [ ]:
# Removed
# Download in https://www.kaggle.com/c/cassava-leaf-disease-classification

In [ ]:
# Create folder for each class
!mkdir '/content/data/train_images/Bacterial Blight'
!mkdir '/content/data/train_images/Brown Streak Disease'
!mkdir '/content/data/train_images/Green Mottle'
!mkdir '/content/data/train_images/Mosaic Disease'
!mkdir '/content/data/train_images/Healthy'
# Create FOlder for checkpoint
!mkdir '/content/checkpoint'

In [ ]:
# Move each image to folder class
import pandas as pd
import shutil

IMG_DIR = '/content/data/train_images/'

df = pd.read_csv('/content/data/train.csv')
df['path'] = IMG_DIR + df['image_id']
df.replace({'label':{0: "Bacterial Blight",
                     1: "Brown Streak Disease",
                     2: "Green Mottle",
                     3: "Mosaic Disease",
                     4: "Healthy"}}, inplace=True)

for idx, row in df.iterrows():
    try:
        shutil.move(row['path'], IMG_DIR+row['label'])
    except:
        pass

## Code

In [ ]:
!pip install tflite-model-maker -q

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tflite_model_maker as tfl
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

def set_seed(SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

set_seed(2021)

IMG_DIR = '/content/data/train_images/'
CKPT_DIR = '/content/checkpoint'

In [ ]:
#@title
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
def plot_eval(model, test_data):
    plt.figure(figsize=(20, 20))
    predicts = model.predict_top_k(test_data)
    for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(25)):
        ax = plt.subplot(5, 5, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image.numpy(), cmap=plt.cm.gray)

        predict_label = predicts[i][0][0]
        true_label = test_data.index_to_label[label.numpy()]
        color = get_label_color(predict_label,
                                test_data.index_to_label[label.numpy()])
        ax.xaxis.label.set_color(color)
        plt.title('True: %s' % true_label)
        plt.xlabel('Pred: %s' % predict_label)
    plt.show()

## Create Dataset

In [ ]:
data = tfl.ImageClassifierDataLoader.from_folder(IMG_DIR)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

INFO:tensorflow:Load image with size: 21397, num_label: 5, labels: Bacterial Blight, Brown Streak Disease, Green Mottle, Healthy, Mosaic Disease.


## Finet Tuning efficientnet_lite3

In [ ]:
tf.keras.backend.clear_session()
fine_mdl0 = tfl.image_classifier.create(train_data=train_data,
                                       model_spec='efficientnet_lite3',
                                       validation_data=validation_data,
                                       epochs=15,
                                       learning_rate=5e-4,
                                       warmup_steps=2*534,
                                       train_whole_model=True,
                                       model_dir=CKPT_DIR,
                                       shuffle=True)

_loss, _accuracy = fine_mdl0.evaluate(test_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              6992768   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 6,999,173
Trainable params: 6,999,173
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15


534/534 [==============================] - 418s 772ms/step - loss: 1.0283 - accuracy: 0.6866 - val_loss: 0.7884 - val_accuracy: 0.7988
Epoch 2/15
534/534 [==============================] - 410s 768ms/step - loss: 0.7891 - accuracy: 0.8082 - val_loss: 0.7552 - val_accuracy: 0.8149
Epoch 3/15
534/534 [==============================] - 410s 769ms/step - loss: 0.7359 - accuracy: 0.8357 - val_loss: 0.7251 - val_accuracy: 0.8395
Epoch 4/15
534/534 [==============================] - 408s 764ms/step - loss: 0.7085 - accuracy: 0.8521 - val_loss: 0.7431 - val_accuracy: 0.8314
Epoch 5/15
534/534 [==============================] - 413s 774ms/step - loss: 0.6868 - accuracy: 0.8616 - val_loss: 0.7342 - val_accuracy: 0.8366
Epoch 6/15
534/534 [==============================] - 411s 771ms/step - loss: 0.6683 - accuracy: 0.8698 - val_loss: 0.7190 - val_accuracy: 0.8461
Epoch 7/15
534/534 [==============================] - 412s 772ms/step - loss: 0.6427 - accuracy: 0.8816 - val_loss: 0.7073 - val_accura

In [ ]:
tf.keras.backend.clear_session()
fine_mdl1 = tfl.image_classifier.create(train_data=train_data,
                                       model_spec='efficientnet_lite3',
                                       validation_data=validation_data,
                                       epochs=15,
                                       learning_rate=5e-4,
                                       warmup_steps=2*534,
                                       train_whole_model=True,
                                       dropout_rate=0,
                                       use_augmentation=True,
                                       model_dir=CKPT_DIR,
                                       shuffle=True)

_loss, _accuracy = fine_mdl1.evaluate(test_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              6992768   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 6,999,173
Trainable params: 6,999,173
Non-trainable params: 0
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


...

INFO:tensorflow:Use customized resize method bilinear


Epoch 1/15


534/534 [==============================] - 385s 713ms/step - loss: 1.0358 - accuracy: 0.6764 - val_loss: 0.8185 - val_accuracy: 0.7884
Epoch 2/15
534/534 [==============================] - 384s 720ms/step - loss: 0.8141 - accuracy: 0.7926 - val_loss: 0.7581 - val_accuracy: 0.8224
Epoch 3/15
534/534 [==============================] - 385s 721ms/step - loss: 0.7643 - accuracy: 0.8190 - val_loss: 0.7225 - val_accuracy: 0.8310
Epoch 4/15
534/534 [==============================] - 382s 715ms/step - loss: 0.7428 - accuracy: 0.8291 - val_loss: 0.7106 - val_accuracy: 0.8509
Epoch 5/15
534/534 [==============================] - 384s 719ms/step - loss: 0.7349 - accuracy: 0.8370 - val_loss: 0.7269 - val_accuracy: 0.8447
Epoch 6/15
534/534 [==============================] - 387s 725ms/step - loss: 0.7230 - accuracy: 0.8433 - val_loss: 0.7122 - val_accuracy: 0.8409
Epoch 7/15
534/534 [==============================] - 390s 731ms/step - loss: 0.7123 - accuracy: 0.8470 - val_loss: 0.7112 - val_accura

INFO:tensorflow:Use customized resize method bilinear


67/67 [==============================] - 48s 165ms/step - loss: 0.6384 - accuracy: 0.8813


In [ ]:
EXPORT_DIR = '/content/drive/MyDrive/Cassava/TFLite_0.8813/'
fine_mdl1.export(export_dir=EXPORT_DIR, export_format=tfl.ExportFormat.TFLITE)
fine_mdl1.export(export_dir=EXPORT_DIR, export_format=tfl.ExportFormat.LABEL)
fine_mdl1.export(export_dir=EXPORT_DIR, export_format=tfl.ExportFormat.SAVED_MODEL)

INFO:tensorflow:Assets written to: /tmp/tmp_saq22kq/assets


INFO:tensorflow:Assets written to: /tmp/tmp_saq22kq/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp6_kx6ert/labels.txt.


INFO:tensorflow:Saving labels in /tmp/tmp6_kx6ert/labels.txt.


INFO:tensorflow:Saving labels in /content/drive/MyDrive/Cassava/TFLite_0.8813/labels.txt.


INFO:tensorflow:Saving labels in /content/drive/MyDrive/Cassava/TFLite_0.8813/labels.txt.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Cassava/TFLite_0.8813/saved_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Cassava/TFLite_0.8813/saved_model/assets


# **We got 88% accuracy in test data**